## 1. Importación de librerías

## 1. Importación de Librerías

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
import cv2
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

ModuleNotFoundError: No module named 'numpy'

## 2. Configuración de Rutas

In [ ]:
ruta_base = Path(r'e:\06. Sexto Ciclo\01. Machine Learning\07. Workspace\16S03. Proyecto 03\P3-EcoSort')
ruta_train = ruta_base / 'data' / 'preprocessed' / 'train'
ruta_val = ruta_base / 'data' / 'preprocessed' / 'val'
ruta_figuras = ruta_base / 'result' / 'figures'

ruta_figuras.mkdir(parents=True, exist_ok=True)

clases = ['general', 'paper', 'plastic']

## 3. Estructura del Dataset

### 3.1 Distribución de Clases

In [ ]:
def contar_imagenes(ruta):
    conteo = {}
    for clase in clases:
        ruta_clase = ruta / clase
        if ruta_clase.exists():
            archivos = list(ruta_clase.glob('*'))
            conteo[clase] = len([f for f in archivos if f.is_file()])
        else:
            conteo[clase] = 0
    return conteo

conteo_train = contar_imagenes(ruta_train)
conteo_val = contar_imagenes(ruta_val)

df_distribucion = pd.DataFrame({
    'Clase': clases,
    'Train': [conteo_train[c] for c in clases],
    'Validación': [conteo_val[c] for c in clases],
    'Total': [conteo_train[c] + conteo_val[c] for c in clases]
})

df_distribucion['Porcentaje Train (%)'] = (df_distribucion['Train'] / df_distribucion['Train'].sum() * 100).round(2)
df_distribucion['Porcentaje Total (%)'] = (df_distribucion['Total'] / df_distribucion['Total'].sum() * 100).round(2)

df_distribucion

### 3.2 Visualización de Distribución de Clases

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(df_distribucion['Clase'], df_distribucion['Train'], alpha=0.8, label='Train', color='#1f77b4')
axes[0].bar(df_distribucion['Clase'], df_distribucion['Validación'], bottom=df_distribucion['Train'], 
           alpha=0.8, label='Validación', color='#ff7f0e')
axes[0].set_xlabel('Clase', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Número de Imágenes', fontsize=12, fontweight='bold')
axes[0].set_title('Distribución de Imágenes por Clase', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(axis='y', alpha=0.3)

for i, (clase, train, val) in enumerate(zip(df_distribucion['Clase'], df_distribucion['Train'], df_distribucion['Validación'])):
    axes[0].text(i, train + val + 20, f'{train + val}', ha='center', fontsize=11, fontweight='bold')

colores = ['#2ecc71', '#e74c3c', '#3498db']
axes[1].pie(df_distribucion['Total'], labels=df_distribucion['Clase'], autopct='%1.1f%%', 
           startangle=90, colors=colores, textprops={'fontsize': 12, 'fontweight': 'bold'})
axes[1].set_title('Proporción de Clases en el Dataset Total', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(ruta_figuras / '01_eda_01_distribucion_clases.svg', format='svg', bbox_inches='tight')
plt.show()

### 3.3 Análisis de Tamaños y Resoluciones

In [ ]:
def analizar_dimensiones_imagenes(ruta, muestra=100):
    dimensiones = []
    aspectos = []
    formatos = []
    
    for clase in clases:
        ruta_clase = ruta / clase
        archivos = list(ruta_clase.glob('*'))[:muestra]
        
        for archivo in archivos:
            if archivo.is_file():
                try:
                    img = Image.open(archivo)
                    ancho, alto = img.size
                    dimensiones.append((ancho, alto))
                    aspectos.append(ancho / alto)
                    formatos.append(img.format)
                except:
                    pass
    
    return dimensiones, aspectos, formatos

dimensiones_train, aspectos_train, formatos_train = analizar_dimensiones_imagenes(ruta_train, muestra=200)

anchos = [d[0] for d in dimensiones_train]
altos = [d[1] for d in dimensiones_train]

df_dimensiones = pd.DataFrame({
    'Métrica': ['Ancho Mínimo', 'Ancho Máximo', 'Ancho Promedio', 'Ancho Mediana',
                'Alto Mínimo', 'Alto Máximo', 'Alto Promedio', 'Alto Mediana',
                'Aspecto Mínimo', 'Aspecto Máximo', 'Aspecto Promedio'],
    'Valor': [
        np.min(anchos), np.max(anchos), np.mean(anchos), np.median(anchos),
        np.min(altos), np.max(altos), np.mean(altos), np.median(altos),
        np.min(aspectos_train), np.max(aspectos_train), np.mean(aspectos_train)
    ]
})

df_dimensiones['Valor'] = df_dimensiones['Valor'].round(2)
df_dimensiones

In [ ]:
formatos_counter = Counter(formatos_train)
df_formatos = pd.DataFrame(formatos_counter.items(), columns=['Formato', 'Cantidad'])
df_formatos = df_formatos.sort_values('Cantidad', ascending=False)
df_formatos

### 3.4 Visualización de Dimensiones

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].scatter(anchos, altos, alpha=0.5, s=30, c='#3498db')
axes[0].set_xlabel('Ancho (píxeles)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Alto (píxeles)', fontsize=12, fontweight='bold')
axes[0].set_title('Dispersión de Dimensiones de Imágenes', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

axes[1].hist(aspectos_train, bins=30, alpha=0.7, color='#e74c3c', edgecolor='black')
axes[1].axvline(np.mean(aspectos_train), color='#2ecc71', linestyle='--', linewidth=2, label=f'Media: {np.mean(aspectos_train):.2f}')
axes[1].set_xlabel('Relación de Aspecto (ancho/alto)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Frecuencia', fontsize=12, fontweight='bold')
axes[1].set_title('Distribución de Relación de Aspecto', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(axis='y', alpha=0.3)

axes[2].bar(df_formatos['Formato'], df_formatos['Cantidad'], alpha=0.8, color='#9b59b6', edgecolor='black')
axes[2].set_xlabel('Formato de Imagen', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Cantidad', fontsize=12, fontweight='bold')
axes[2].set_title('Distribución de Formatos de Imagen', fontsize=14, fontweight='bold')
axes[2].grid(axis='y', alpha=0.3)

for i, (formato, cantidad) in enumerate(zip(df_formatos['Formato'], df_formatos['Cantidad'])):
    axes[2].text(i, cantidad + 5, str(cantidad), ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig(ruta_figuras / '01_eda_02_dimensiones_imagenes.svg', format='svg', bbox_inches='tight')
plt.show()

## 4. Visualización Exploratoria

### 4.1 Ejemplos Representativos de Cada Clase

In [ ]:
def mostrar_ejemplos_clase(ruta, clase, num_ejemplos=8):
    ruta_clase = ruta / clase
    archivos = list(ruta_clase.glob('*'))[:num_ejemplos]
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle(f'Ejemplos Representativos - Clase: {clase.upper()}', fontsize=16, fontweight='bold')
    
    for idx, archivo in enumerate(archivos):
        if archivo.is_file():
            img = Image.open(archivo)
            ax = axes[idx // 4, idx % 4]
            ax.imshow(img)
            ax.set_title(f'{img.size[0]}×{img.size[1]} px', fontsize=10, fontweight='bold')
            ax.axis('off')
    
    plt.tight_layout()
    plt.savefig(ruta_figuras / f'01_eda_03_ejemplos_{clase}.svg', format='svg', bbox_inches='tight')
    plt.show()

for clase in clases:
    mostrar_ejemplos_clase(ruta_train, clase, num_ejemplos=8)

### 4.2 Análisis de Variabilidad Visual

In [ ]:
def analizar_variabilidad(ruta, muestra=50):
    estadisticas = []
    
    for clase in clases:
        ruta_clase = ruta / clase
        archivos = list(ruta_clase.glob('*'))[:muestra]
        
        brillos = []
        contrastes = []
        saturaciones = []
        
        for archivo in archivos:
            if archivo.is_file():
                try:
                    img = cv2.imread(str(archivo))
                    if img is not None:
                        img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
                        
                        brillo = np.mean(img_hsv[:, :, 2])
                        brillos.append(brillo)
                        
                        contraste = np.std(img)
                        contrastes.append(contraste)
                        
                        saturacion = np.mean(img_hsv[:, :, 1])
                        saturaciones.append(saturacion)
                except:
                    pass
        
        estadisticas.append({
            'Clase': clase,
            'Brillo Promedio': np.mean(brillos) if brillos else 0,
            'Brillo Std': np.std(brillos) if brillos else 0,
            'Contraste Promedio': np.mean(contrastes) if contrastes else 0,
            'Contraste Std': np.std(contrastes) if contrastes else 0,
            'Saturación Promedio': np.mean(saturaciones) if saturaciones else 0,
            'Saturación Std': np.std(saturaciones) if saturaciones else 0
        })
    
    return pd.DataFrame(estadisticas)

df_variabilidad = analizar_variabilidad(ruta_train, muestra=100)
df_variabilidad = df_variabilidad.round(2)
df_variabilidad

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

x = np.arange(len(clases))
ancho_barra = 0.35

axes[0].bar(x - ancho_barra/2, df_variabilidad['Brillo Promedio'], ancho_barra, 
           label='Promedio', alpha=0.8, color='#f39c12', edgecolor='black')
axes[0].bar(x + ancho_barra/2, df_variabilidad['Brillo Std'], ancho_barra, 
           label='Desviación Estándar', alpha=0.8, color='#e67e22', edgecolor='black')
axes[0].set_xlabel('Clase', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Valor', fontsize=12, fontweight='bold')
axes[0].set_title('Análisis de Brillo por Clase', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(clases)
axes[0].legend(fontsize=10)
axes[0].grid(axis='y', alpha=0.3)

axes[1].bar(x - ancho_barra/2, df_variabilidad['Contraste Promedio'], ancho_barra, 
           label='Promedio', alpha=0.8, color='#3498db', edgecolor='black')
axes[1].bar(x + ancho_barra/2, df_variabilidad['Contraste Std'], ancho_barra, 
           label='Desviación Estándar', alpha=0.8, color='#2980b9', edgecolor='black')
axes[1].set_xlabel('Clase', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Valor', fontsize=12, fontweight='bold')
axes[1].set_title('Análisis de Contraste por Clase', fontsize=14, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(clases)
axes[1].legend(fontsize=10)
axes[1].grid(axis='y', alpha=0.3)

axes[2].bar(x - ancho_barra/2, df_variabilidad['Saturación Promedio'], ancho_barra, 
           label='Promedio', alpha=0.8, color='#e74c3c', edgecolor='black')
axes[2].bar(x + ancho_barra/2, df_variabilidad['Saturación Std'], ancho_barra, 
           label='Desviación Estándar', alpha=0.8, color='#c0392b', edgecolor='black')
axes[2].set_xlabel('Clase', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Valor', fontsize=12, fontweight='bold')
axes[2].set_title('Análisis de Saturación por Clase', fontsize=14, fontweight='bold')
axes[2].set_xticks(x)
axes[2].set_xticklabels(clases)
axes[2].legend(fontsize=10)
axes[2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(ruta_figuras / '01_eda_06_variabilidad_visual.svg', format='svg', bbox_inches='tight')
plt.show()

## 5. Análisis Cuantitativo de Clases

### 5.1 Medición de Desbalance

In [ ]:
total_train = df_distribucion['Train'].sum()
clase_mayoritaria = df_distribucion['Train'].max()
clase_minoritaria = df_distribucion['Train'].min()

ratio_desbalance = clase_mayoritaria / clase_minoritaria

df_desbalance = pd.DataFrame({
    'Métrica': ['Total Train', 'Clase Mayoritaria', 'Clase Minoritaria', 'Ratio de Desbalance'],
    'Valor': [total_train, clase_mayoritaria, clase_minoritaria, ratio_desbalance]
})

df_desbalance['Valor'] = df_desbalance['Valor'].round(2)
df_desbalance

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

colores_barra = ['#2ecc71' if x == clase_mayoritaria else '#e74c3c' if x == clase_minoritaria else '#3498db' 
                 for x in df_distribucion['Train']]

bars = ax.bar(df_distribucion['Clase'], df_distribucion['Train'], alpha=0.8, color=colores_barra, edgecolor='black')

ax.axhline(y=total_train/3, color='orange', linestyle='--', linewidth=2, label=f'Balance Ideal: {int(total_train/3)}')

ax.set_xlabel('Clase', fontsize=12, fontweight='bold')
ax.set_ylabel('Número de Imágenes', fontsize=12, fontweight='bold')
ax.set_title(f'Desbalance de Clases (Ratio: {ratio_desbalance:.2f}:1)', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(axis='y', alpha=0.3)

for i, (clase, cantidad) in enumerate(zip(df_distribucion['Clase'], df_distribucion['Train'])):
    ax.text(i, cantidad + 20, f'{cantidad}\n({df_distribucion["Porcentaje Train (%)"].iloc[i]}%)', 
           ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig(ruta_figuras / '01_eda_07_desbalance_clases.svg', format='svg', bbox_inches='tight')
plt.show()

## 6. Detección de Problemas

### 6.1 Verificación de Imágenes Corruptas

In [ ]:
def verificar_imagenes_corruptas(ruta):
    corruptas = []
    totales = 0
    
    for clase in clases:
        ruta_clase = ruta / clase
        archivos = list(ruta_clase.glob('*'))
        
        for archivo in archivos:
            if archivo.is_file():
                totales += 1
                try:
                    img = Image.open(archivo)
                    img.verify()
                except:
                    corruptas.append({'Clase': clase, 'Archivo': archivo.name})
    
    return corruptas, totales

corruptas_train, total_train_check = verificar_imagenes_corruptas(ruta_train)
corruptas_val, total_val_check = verificar_imagenes_corruptas(ruta_val)

df_corruptas = pd.DataFrame({
    'Conjunto': ['Train', 'Validación', 'Total'],
    'Imágenes Totales': [total_train_check, total_val_check, total_train_check + total_val_check],
    'Imágenes Corruptas': [len(corruptas_train), len(corruptas_val), len(corruptas_train) + len(corruptas_val)],
    'Porcentaje Corruptas (%)': [
        (len(corruptas_train) / total_train_check * 100) if total_train_check > 0 else 0,
        (len(corruptas_val) / total_val_check * 100) if total_val_check > 0 else 0,
        ((len(corruptas_train) + len(corruptas_val)) / (total_train_check + total_val_check) * 100) 
        if (total_train_check + total_val_check) > 0 else 0
    ]
})

df_corruptas['Porcentaje Corruptas (%)'] = df_corruptas['Porcentaje Corruptas (%)'].round(2)
df_corruptas

### 6.2 Verificación de Dimensiones Inconsistentes

In [ ]:
def verificar_dimensiones_extremas(ruta, umbral_min=50, umbral_max=5000):
    problematicas = []
    
    for clase in clases:
        ruta_clase = ruta / clase
        archivos = list(ruta_clase.glob('*'))
        
        for archivo in archivos:
            if archivo.is_file():
                try:
                    img = Image.open(archivo)
                    ancho, alto = img.size
                    
                    if ancho < umbral_min or alto < umbral_min or ancho > umbral_max or alto > umbral_max:
                        problematicas.append({
                            'Clase': clase,
                            'Archivo': archivo.name,
                            'Ancho': ancho,
                            'Alto': alto,
                            'Problema': 'Muy pequeña' if (ancho < umbral_min or alto < umbral_min) else 'Muy grande'
                        })
                except:
                    pass
    
    return problematicas

problematicas_train = verificar_dimensiones_extremas(ruta_train)
problematicas_val = verificar_dimensiones_extremas(ruta_val)

df_problemas = pd.DataFrame({
    'Conjunto': ['Train', 'Validación', 'Total'],
    'Dimensiones Extremas': [len(problematicas_train), len(problematicas_val), 
                            len(problematicas_train) + len(problematicas_val)]
})

df_problemas

## 7. Conclusiones del EDA

### Hallazgos Principales:

**1. Distribución del Dataset:**
- El dataset contiene 3 clases: general, paper y plastic
- Existe un desbalance notable entre las clases
- La clase plastic es la minoritaria, lo que requerirá técnicas de balanceo

**2. Características de las Imágenes:**
- Las imágenes tienen dimensiones variables y diversos formatos
- Se requiere normalización de tamaños para el modelado
- La variabilidad en brillo, contraste y saturación sugiere necesidad de data augmentation

**3. Calidad del Dataset:**
- No se detectaron imágenes corruptas significativas
- Las dimensiones son generalmente consistentes
- El dataset está en buenas condiciones para el entrenamiento

**4. Recomendaciones para Feature Engineering:**
- Aplicar data augmentation para balancear clases y aumentar variabilidad
- Normalizar dimensiones a tamaño fijo (224x224 o 128x128)
- Extraer características HOG y de color para modelos no neuronales
- Considerar técnicas de oversampling para la clase minoritaria
- Aplicar normalización de píxeles para mejorar convergencia de modelos